# Maria SQL Data Exploration Notebook

An example notebook which can be used to perform data analysis using an MariaDB SQL client and Pandas dataframes.

In [ ]:
import json
import os
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import paramiko
import pymysql
import sqlfluff
from box import Box
from dateutil.relativedelta import relativedelta
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

from cupyopt.schema import avro_schema, avro_schema_to_file, infer_df_avro_schema
from cupyopt.validation import df_avro_validate

In [ ]:
"""
config_filepath found below should include yaml data similar to the following
<databasename>:
    ssh_hostname: str
    ssh_port: int
    ssh_username: str
    ssh_key: str
    sql_hostname: str
    sql_port: int
    sql_username: str
    sql_password: str
    database: str
"""

# filepath to configuration file in yaml format
config_filepath = "config.yaml"
database_name = "databasename"

In [ ]:
# open configuration file containing relevant connection details for oracle database
config = Box.from_yaml(filename=config_filepath)[database_name]
# create paramiko private key for use with ssh cxn
pkey = paramiko.RSAKey.from_private_key_file(config["ssh_key"])

In [ ]:
# inspired by following SO post -> https://stackoverflow.com/questions/21903411/enable-python-to-connect-to-mysql-via-ssh-tunnelling
def ssh_tunneled_sql_query(
    sql_stmt: str,
    config: Box,
    ssh_pkey: paramiko.pkey.PKey,
) -> pd.DataFrame:
    """
    Makes SSH-tunneled SQL Query, receiving and returning data as pd.DataFrame
    """
    with SSHTunnelForwarder(
        (config.ssh_hostname, config.ssh_port),
        ssh_username=config.ssh_username,
        ssh_pkey=pkey,
        remote_bind_address=(config.sql_hostname, config.sql_port),
    ) as tunnel:
        conn = pymysql.connect(
            host=config.sql_hostname,
            user=config.sql_username,
            passwd=config.sql_password,
            port=tunnel.local_bind_port,
        )
        data = pd.read_sql_query(sql_stmt, conn)
        conn.close()

    return data

In [ ]:
sql_stmt = """
select version()
"""
ssh_tunneled_sql_query(sql_stmt=sql_stmt, ssh_pkey=pkey, config=config)

In [ ]:
sql_stmt = """
select *
from information_schema.schemata
order by schema_name;
"""
ssh_tunneled_sql_query(sql_stmt=sql_stmt, ssh_pkey=pkey, config=config)